In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.


Importing impoertant libraries

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D,Dense, Flatten, GlobalAveragePooling2D, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.python.keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from os.path import join, exists, expanduser
from os import listdir, makedirs
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
import keras as k

In [ ]:
data_dir = '../input'
labels = pd.read_csv(join(data_dir, 'labels.csv'))
Test_submission=pd.read_csv(join(data_dir, 'sample_submission.csv'))

In [ ]:
Test_submission.head(5)

In [ ]:
Test_submission.shape

In [ ]:
labels.head(5)

In [ ]:
labels.breed.head(5)

In [ ]:
labels.pivot_table(index='breed',aggfunc=len).sort_values('id',ascending=False)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 14))
labels['breed'].value_counts().plot(kind='bar')
plt.show()

In [ ]:
im_size = 224

x_train = []
y_train = []

i = 0 
for f, breed in tqdm(labels.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = labels.breed[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

In [ ]:
x_test = []

for f in tqdm(Test_submission['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

In [ ]:
x_train_new=np.array(x_train)

x_test_new=np.array(x_test)

In [ ]:
x_train_new.shape,x_test_new.shape

In [ ]:
num_class=120
le=preprocessing.LabelEncoder()
Y=le.fit_transform(y_train)
Y=k.utils.to_categorical(Y, num_class)

In [ ]:
Y.shape

In [ ]:
pretrained_model = ResNet50(weights='imagenet',include_top=False, pooling='avg')

In [ ]:
pretrained_model.layers.pop()

In [ ]:
for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
model = k.models.Sequential()
model.add(pretrained_model)
model.add(k.layers.Dense(512, activation='relu'))
model.add(k.layers.Dropout(.5))
model.add(k.layers.Dense(num_class, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='sgd', metrics=['accuracy'])
callbacks_list = [k.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()


In [ ]:
print('Input Shape = {}'.format(model.layers[0].input_shape))
print('Shape Shape = {}'.format(model.layers[-1].output_shape))

In [ ]:
model_log=model.fit(
    x=x_train_new, y=Y, batch_size=32,
    epochs=25, verbose=1,validation_split=.2
     
)

In [ ]:
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(model_log.history['acc'])
#plt.plot(model_log.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

In [ ]:
prediction=model.predict(x_test_new, verbose=1)

In [ ]:
sub = pd.DataFrame(prediction)

In [ ]:
Test_submission1=Test_submission

In [ ]:
sub.insert(0, 'id', Test_submission1['id'])
sub.head(5)

In [ ]:
col_names = Test_submission1.columns.values
sub.columns = col_names
sub.head(5)

In [ ]:
sub.to_csv('submission1.csv', index = False)